# How to freeze orbitals (with the H₂O molecule as an example)

In this guide, we apply Entanglement Forging to compute the energy of a $\mathrm{H}_2\mathrm{O}$ molecule. We reduce the number of orbitals in the problem, in turn reducing the number of qubits needed in each circuit, following the logic given in the [explanatory material](../explanation/index.rst).

In [ ]:
import numpy as np

from qiskit.circuit import QuantumCircuit, Parameter
from qiskit.algorithms.optimizers import COBYLA
from qiskit_nature.second_q.drivers import PySCFDriver

from circuit_knitting.forging import (
    EntanglementForgingAnsatz,
    EntanglementForgingGroundStateSolver,
)
from circuit_knitting.utils import reduce_bitstrings

### Instantiate the `ElectronicStructureProblem`

Next, we set up the $\mathrm{H}_2\mathrm{O}$ molecule, specify the driver and converter, and instantiate an `ElectronicStructureProblem`.

In [2]:
radius_1 = 0.958  # position for the first H atom
radius_2 = 0.958  # position for the second H atom
thetas_in_deg = 104.478  # bond angles.

H1_x = radius_1
H2_x = radius_2 * np.cos(np.pi / 180 * thetas_in_deg)
H2_y = radius_2 * np.sin(np.pi / 180 * thetas_in_deg)

# Create an ElectronicStructureProblem from the molecule using PySCFDriver
driver = PySCFDriver(
    f"O 0.0 0.0 0.0; H {H1_x} 0.0 0.0; H {H2_x} {H2_y} 0.0", basis="sto6g"
)
driver.run()
problem = driver.to_problem()

### Prepare the bitstrings and the ansatz

In [3]:
theta = Parameter("θ")

hop_gate = QuantumCircuit(2, name="Hop gate")
hop_gate.h(0)
hop_gate.cx(1, 0)
hop_gate.cx(0, 1)
hop_gate.ry(-theta, 0)
hop_gate.ry(-theta, 1)
hop_gate.cx(0, 1)
hop_gate.h(0)

hop_gate.draw()

┌───┐┌───┐     ┌────────────┐     ┌───┐
q_0: ┤ H ├┤ X ├──■──┤ Ry(-1.0*θ) ├──■──┤ H ├
     └───┘└─┬─┘┌─┴─┐├────────────┤┌─┴─┐└───┘
q_1: ───────■──┤ X ├┤ Ry(-1.0*θ) ├┤ X ├─────
               └───┘└────────────┘└───┘

Create the ansatz with a reduced set of bitstrings.

In [4]:
theta_1, theta_2, theta_3, theta_4 = (
    Parameter("θ1"),
    Parameter("θ2"),
    Parameter("θ3"),
    Parameter("θ4"),
)

circuit_u = QuantumCircuit(5)
circuit_u.append(hop_gate.to_gate({theta: theta_1}), [0, 1])
circuit_u.append(hop_gate.to_gate({theta: theta_2}), [3, 4])
circuit_u.append(hop_gate.to_gate({theta: 0}), [1, 4])
circuit_u.append(hop_gate.to_gate({theta: theta_3}), [0, 2])
circuit_u.append(hop_gate.to_gate({theta: theta_4}), [3, 4])

# Set our bitstrings, and then reduce the chosen orbitals
orbitals_to_reduce = [0, 3]
bitstrings_u = [(1, 1, 1, 1, 1, 0, 0), (1, 0, 1, 1, 1, 0, 1), (1, 0, 1, 1, 1, 1, 0)]
reduced_bitstrings = reduce_bitstrings(bitstrings_u, orbitals_to_reduce)

ansatz = EntanglementForgingAnsatz(circuit_u=circuit_u, bitstrings_u=reduced_bitstrings)

ansatz.circuit_u.draw()

┌───────────────┐                ┌───────────────┐
q_0: ┤0              ├────────────────┤0              ├
     │  Hop gate(θ1) │┌──────────────┐│               │
q_1: ┤1              ├┤0             ├┤  Hop gate(θ3) ├
     └───────────────┘│              ││               │
q_2: ─────────────────┤              ├┤1              ├
     ┌───────────────┐│  Hop gate(0) │├───────────────┤
q_3: ┤0              ├┤              ├┤0              ├
     │  Hop gate(θ2) ││              ││  Hop gate(θ4) │
q_4: ┤1              ├┤1             ├┤1              ├
     └───────────────┘└──────────────┘└───────────────┘

Create the ``EntanglementForgingGroundStateSolver``, and specify which orbitals to reduce in the observable.

In [5]:
optimizer = COBYLA(maxiter=100)

solver = EntanglementForgingGroundStateSolver(
    ansatz=ansatz,
    optimizer=optimizer,
    orbitals_to_reduce=orbitals_to_reduce,
)

From here, the problem can be solved following the same steps as in the [tutorials](../tutorials/index.rst).

In [6]:
import qiskit.tools.jupyter  # noqa: F401

%qiskit_version_table

### This code is a Qiskit project.

© Copyright IBM 2022.

This code is licensed under the Apache License, Version 2.0. You may
obtain a copy of this license in the LICENSE.txt file in the root directory
of this source tree or at http://www.apache.org/licenses/LICENSE-2.0.

Any modifications or derivative works of this code must retain this
copyright notice, and modified files need to carry a notice indicating
that they have been altered from the originals.